In [1]:
%pip install fastapi pydantic toml

Note: you may need to restart the kernel to use updated packages.


In [2]:
# from rs import skill_check

# def test_skill_check(metadata):
#     print(skill_check("sanity", None, metadata))

# metadata = {
#     "character": {"sanity":30},
#     "symptom":[
#         "incoherent speech",
#         "uncontrollable twitching, trembling",
#         "delusions of persecution",
#         "strange appetites (dirt, clay, etc...)",
#         "scratches, punches, and bruises all over the body"
#     ]
# }

# test_skill_check(metadata)
# test_skill_check(metadata)
# test_skill_check(metadata)
# test_skill_check(metadata)
# test_skill_check(metadata)
# test_skill_check(metadata)
# test_skill_check(metadata)
# test_skill_check(metadata)
# test_skill_check(metadata)
# test_skill_check(metadata)
# test_skill_check(metadata)
# test_skill_check(metadata)

# print(metadata["character"])


In [3]:
import re
import tomllib
import functools
from typing import Literal

import ipywidgets as widgets
from IPython.display import display

from dotenv import load_dotenv

load_dotenv()

from rs import Message, Config, chat, skill_check
import d20

action_pattern = re.compile(r"(\d+). *(.+)")
skill_pattern = re.compile(r"[\w_]+")
skill_difficulty_pattern = re.compile(r"(easy|medium|hard)")

def next_round(b):
    global current_round, initial_messages
    current_round += 1
    initial_messages = config.initial_messages(current_round, config.metadata)

    print(initial_messages[-1].content)
    display(get_controls(initial_messages[-1]))

def get_user_message_tail():
    count = 0
    for msg in initial_messages:
        if msg.role == "user":
            count += 1

    is_final =  ", FINAL ROUND" if  current_round == len(config.rounds) else ""
    
    # print(config.metadata["character"])
    
    if "tags" in config.metadata["character"] and "insane" in config.metadata["character"]["tags"]:
        return f"TURN: {count + 1} {is_final} (I'm insane!)"
    
    return f"TURN: {count + 1} {is_final}"
    

def do_chat(role:Literal["user", "system", "assistant"], content: str):
    print(f"{content}\n\n")

    initial_messages.append(Message(role=role, content=content))
    msg = chat(initial_messages, lambda delta: print(delta, end=""))

    initial_messages.append(msg)
    display(get_controls(msg))


def on_skill(b, skill:str, difficulty:str):
    # print(config.metadata["character"])
    content = skill_check(skill, difficulty, config.metadata)
    content += f"\n{get_user_message_tail()}"
    do_chat("user", content)

def on_action(b, index:int):
    content = f"I select option {index}. {get_user_message_tail()}"
    do_chat("user", content)

def on_custom_input(b, input:widgets.Text):
    content = f"{input.value}. {get_user_message_tail()}"
    do_chat("user", content)

def on_custom_action(b):
    hbox = widgets.HBox(layout=widgets.Layout(width="100%"))
    action_text = widgets.Text(layout=widgets.Layout(width="100%"))
    submit = widgets.Button(description="SUBMIT", layout=widgets.Layout(width="fit-content"))
    submit.on_click(functools.partial(on_custom_input, input=action_text))
    hbox.children += (action_text, submit) 
    display(hbox)

def get_controls(msg:Message) -> widgets.Box:
    dict = msg.dict()
    buttons = []
    if "possible actions" in dict.keys():
        matches = action_pattern.finditer(dict["possible actions"])
        for match in matches:
            btn = widgets.Button(description=match.group(1), tooltip=match.group(2), layout=widgets.Layout(width="auto"))
            btn.on_click(functools.partial(on_action, index=int(match.group(1))))
            buttons.append(btn)
    elif "skill" in dict.keys() and "difficulty" in dict.keys():
        match = skill_pattern.match(dict["skill"])
        skill = match.group(0)
        match = skill_difficulty_pattern.match(dict["difficulty"])
        difficulty = match.group(0)
        skill_button = widgets.Button(description=f"{skill.upper()} [{difficulty.upper()}]", layout=widgets.Layout(width="auto"))
        skill_button.on_click(functools.partial(on_skill, skill=skill, difficulty=difficulty))
        buttons.append(skill_button)
    elif "summary" in dict.keys():
        # add summary as a memory to the metadata
        # it will be used in the next round.
        config.metadata["memories"] = []
        config.metadata["memories"].append(dict["summary"])
        next_btn = widgets.Button(description="NEXT ROUND", layout=widgets.Layout(width="auto"))
        next_btn.on_click(next_round)
        buttons.append(next_btn)
    
    custom_action_button = widgets.Button(description="CUSTOM")
    custom_action_button.on_click(on_custom_action)
    buttons.append(custom_action_button)

    return widgets.HBox(buttons)

# open config file
config_name = "data/config_skillcheck_test copy.toml"
config: Config | None = None
with open(config_name, "rb") as f:
    data = tomllib.load(f)
    config = Config(**data)

current_round = 0
initial_messages = config.initial_messages(current_round, config.metadata)

print(initial_messages[-1].content)
display(get_controls(initial_messages[-1]))

You arrived at an old self-storage facility by invitation from the manager, who has an storage room up for auction due to the death of its owner. It is not your first rodeo with these forms of auctions, like a box of chocolate, no one knows of the potential treasure that hides behind the cold roller shutter. As you and other interested buyers gathered around the auctioneer, a worker carrying a bolt cutter snapped off the rusty lock, unveiling a storage room cluttered with various old furniture and antiquities. Without any opportunity for the bidders to take a clear look, the auctioneer began the bid at $50. 

What is your bid?

possible actions:
1. $100.
2. $150.
3. $200.


I select option 3. TURN: 1 


The auctioneer nods at your bid, "We have $200 from the gentleman over here!" The atmosphere becomes tense as other bidders weigh their options. After a moment of silence, a lady in the back raises her hand, "I'll go $250."

possible actions:
1. Bid $300.
2. Bid $350.
3. Stay at $200 and see if anyone else bids higher.

I select option 2. TURN: 2 


"Ah, we have $350 from the gentleman!" the auctioneer announces with a hint of excitement in his voice. The crowd murmurs, and you can see a few bidders shaking their heads, stepping back from the competition. The lady who previously bid $250 hesitates for a moment, then reluctantly raises her hand, "I'll go $400."

possible actions:
1. Bid $450, all in.
2. Stay at $350 and hope for the best.

I select option 1. TURN: 3 


With a determined look, you call out, "I'll bid $450, all in." The auctioneer's eyes widen slightly, impressed by your bold move. The crowd falls silent, waiting to see if anyone dares to challenge your bid. The lady who bid $400 looks around, perhaps hoping someone else would jump in, but eventually, she shakes her head and steps back.

The auctioneer waits a moment longer for any further bids, then raises his gavel, "Going once, going twice... and sold to the gentleman for $450!" Applause breaks out among the crowd as the auctioneer congratulates you on your purchase.

summary: Kyle Jackson wins the self-storage auction with a bold bid of $450, spending all his money but potentially securing valuable or mysterious contents within the storage room. The round is completed with Kyle's successful bid.

You follow the manager through the myriad hallways to his dimly lit office, where the furnishings and appliances remain unchanged since the 90’s. He often reminds you that the facility is in fact even older, and has been here since the 60’s. You made a quick phone call to Alex, a man you owe money to, and explained to him that you will pay him back in a few days once you have it, to which he responded with distasteful threats. The manager took out an old tin box and placed it on his desk and reached towards you with an open palm, gesturing for you to pay up, which you obliged. Upon handing you a key for your newly owned storage room, he urged you to watch some old security footage.

Through his CRT screen, he explains to you that the previous owner visited the storage room every day for as long as he can remember. What’s even stranger is that he would hop 3 times towards the storage before entering, and would always bring a paper bag with him, and it would always be empty when he leave

...

I select option 1. TURN: 6 


Compelled by the soft thud coming from the large trunk, you cautiously approach it. The trunk is old, with metal bindings and a heavy padlock securing it shut. However, you notice that the padlock is not fully latched, as if it was hastily closed and not secured properly.

With a sense of trepidation, you lift the lid of the trunk. Inside, you find an assortment of bizarre and macabre items: a collection of ancient-looking tomes bound in unfamiliar materials, several small, intricately carved statues that depict grotesque figures, and a folded piece of old, yellowed parchment on top.

You carefully unfold the parchment, revealing a hand-drawn map. The map appears to detail a series of tunnels or catacombs, with various symbols and notes scrawled in the margins. One area is marked with a symbol that eerily resembles the one carved on the puzzle box you found earlier.

As you're examining the map, a sudden chill fills the room, and the dim light from the hal

I select option 2. TURN: 7 


Feeling a connection between the symbol on the map and the puzzle box, you decide to use the clue from the map to attempt to solve the puzzle box. You retrieve the box from the crate, the symbols carved into its surface seeming to pulsate in the dim light of the storage room. With the haunting melody from the phonograph filling the space, you focus on the symbols, comparing them to the one marked on the map.

After a few moments of concentration, you begin to manipulate the symbols on the box, aligning them in a sequence that matches the arrangement suggested by the map. As you complete the sequence, you hear a soft click from the box. The lid springs open slightly, revealing a small, dark compartment inside.

Within the compartment lies a single, ancient-looking key. The key is made of a heavy metal, its bow intricately designed to resemble the same grotesque figure depicted on some of the statues within the trunk. The teeth of the key are unlike any you'

I select option 3. TURN: 8 


Driven by the unnerving melody and the mysterious circumstances, you decide to investigate the source of the phonograph playing on its own. As you approach the phonograph, the music seems to swell, filling the storage room with its haunting tones. The phonograph itself appears ancient, its wood frame worn and its metal components tarnished with age. Yet, it plays as if it were brand new, the sound clear and disturbingly captivating.

You examine the phonograph closely, looking for any mechanism or device that could have caused it to start playing without human intervention. However, you find nothing out of the ordinary; it's simply an old, hand-cranked phonograph. The record spins steadily, the needle tracing the grooves with precision.

Feeling a growing sense of unease, you reach out and stop the phonograph once more, halting the melody. The silence that follows is almost deafening, the sudden absence of sound making the storage room feel even more isola

I select option 1. TURN: 9 


Compelled by the faint, glowing light, you cautiously approach the antique mirror. The frame is ornate, carved with intricate designs that seem to twist and writhe in the dim light of the storage room. As you draw closer, the source of the glow becomes apparent: the mirror's surface itself seems to shimmer with a light that doesn't reflect the room around you.

Standing before the mirror, you catch your reflection—or rather, a distorted version of it. The light within the mirror casts strange shadows across your features, making you appear older, your eyes hollow and haunted. The room reflected in the mirror is similar to the storage room, yet different; it's darker, the shadows deeper, and you can see shapes moving just at the edge of your vision.

As you watch, transfixed, the shapes begin to coalesce into a figure standing behind your reflection. It's tall and indistinct, its features blurred and shifting, but there's an unmistakable sense of malevolenc

I select option 1. TURN: 10 


With a mixture of fear and curiosity, you decide to try to communicate with the figure in the mirror. You steady your voice and ask, "Who are you? What do you want?"

The figure stops its advance, its form becoming slightly clearer. It tilts its head, as if curious, and then, in a voice that seems to come from all around you, it speaks. "Seeker," it whispers, the word echoing as if spoken in a vast, empty chamber. "The key... it unlocks the gate... to knowledge... to power... but also... to despair."

The figure's warning is cryptic, its intentions unclear. Yet, there's a sincerity in its voice that chills you to the bone. It seems to be both cautioning you and compelling you towards something inevitable.

As it speaks, the glow in the mirror intensifies, casting an otherworldly light across the storage room. The figure's form begins to fade, dissolving into the light until it's gone, leaving only your own reflection looking back at you, albeit shaken and